# 四、神经网络优化


1. 现在的神经元模型添加了激活函数f和偏置B
  * 激活函数：引入非线性激活因素， 提高模型的表达力。常用的激活函数：
    * 激活函数 relu: 在 Tensorflow 中， 用 tf.nn.relu()表示
    * 激活函数 sigmoid：在 Tensorflow 中， 用 tf.nn.sigmoid()表示
    * 激活函数 tanh：在 Tensorflow 中， 用 tf.nn.tanh()表示
2. 神经网络优化的参数：神经网络中所有参数 w 的个数 + 所有参数 b 的个数
3. 神经网络的优化从损失函数loss,学习率learning_rate，滑动平均ema、正则化regu;arization这几个方面出发
  * 损失函数loss:预测值(y)与已知值(y_)的差距,主流的损失函数有
      * mes
      * 自定义
      * 交叉熵ce(Cross Entropy)
  * 

## 1. 损失函数
* 采用官方提供的均方误差

In [1]:
# coding:utf-8
# 去掉警告
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
# 预测多或预测少的影响一样
# 利润成本一样
# 0导入模块，生成数据集
import tensorflow as tf
import numpy as np
BATCH_SIZE = 8
SEED = 23455   # 实际中可以不写的

# 生成数据
rdm = np.random.RandomState(SEED)
X = rdm.rand(32,2)
Y_ = [[x1+x2+(rdm.rand()/10.0-0.05)] for (x1, x2) in X]

# 1定义神经网络的输入、参数和输出，定义前向传播过程。
x = tf.placeholder(tf.float32, shape=(None, 2))
y_ = tf.placeholder(tf.float32, shape=(None, 1))
w1= tf.Variable(tf.random_normal([2, 1], stddev=1, seed=1))
y = tf.matmul(x, w1)

# 2定义损失函数及反向传播方法。
# 定义损失函数为MSE,反向传播方法为梯度下降。
loss_mse = tf.reduce_mean(tf.square(y_ - y))
train_step = tf.train.GradientDescentOptimizer(0.001).minimize(loss_mse)

# 3生成会话，训练STEPS轮
with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    STEPS = 20000
    for i in range(STEPS):
        start = (i*BATCH_SIZE) % 32
        end = (i*BATCH_SIZE) % 32 + BATCH_SIZE
        sess.run(train_step, feed_dict={x: X[start:end], y_: Y_[start:end]})
        if i % 500 == 0:
            print("After %d training steps, w1 is: " % (i))
            print(sess.run(w1), "\n")
    print("Final w1 is: \n", sess.run(w1))


After 0 training steps, w1 is: 
[[-0.80974597]
 [ 1.4852903 ]] 

After 500 training steps, w1 is: 
[[-0.46074435]
 [ 1.641878  ]] 

After 1000 training steps, w1 is: 
[[-0.21939856]
 [ 1.6984766 ]] 

After 1500 training steps, w1 is: 
[[-0.04415595]
 [ 1.7003176 ]] 

After 2000 training steps, w1 is: 
[[0.08942621]
 [1.673328  ]] 

After 2500 training steps, w1 is: 
[[0.19583553]
 [1.6322677 ]] 

After 3000 training steps, w1 is: 
[[0.28375748]
 [1.5854434 ]] 

After 3500 training steps, w1 is: 
[[0.35848638]
 [1.5374471 ]] 

After 4000 training steps, w1 is: 
[[0.4233252]
 [1.4907392]] 

After 4500 training steps, w1 is: 
[[0.48040032]
 [1.4465573 ]] 

After 5000 training steps, w1 is: 
[[0.5311361]
 [1.4054534]] 

After 5500 training steps, w1 is: 
[[0.57653254]
 [1.367594  ]] 

After 6000 training steps, w1 is: 
[[0.6173259]
 [1.3329402]] 

After 6500 training steps, w1 is: 
[[0.65408474]
 [1.3013425 ]] 

After 7000 training steps, w1 is: 
[[0.68726856]
 [1.2726018 ]] 

After 7500 t

## 2. 自定义损失函数 
* 以下这个例子的目标是 ：我们定义单位的酸奶的成本相比酸奶的利润要低，所以如果预测少了的话，会使得总体利润减少，所以生成的模型应该多预测一些，我们就会多生产一些。

In [4]:
# coding:utf-8
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

# 酸奶成本1元， 酸奶利润9元
# 预测少了损失大，故不要预测少，故生成的模型会多预测一些
# 0导入模块，生成数据集
import tensorflow as tf
import numpy as np
BATCH_SIZE = 8
SEED = 23455
COST = 1
PROFIT = 9

rdm = np.random.RandomState(SEED)
X = rdm.rand(32,2)
Y = [[x1+x2+(rdm.rand()/10.0-0.05)] for (x1, x2) in X]

# 1定义神经网络的输入、参数和输出，定义前向传播过程。
x = tf.placeholder(tf.float32, shape=(None, 2))
y_ = tf.placeholder(tf.float32, shape=(None, 1))
w1= tf.Variable(tf.random_normal([2, 1], stddev=1, seed=1))
y = tf.matmul(x, w1)

# 2定义损失函数及反向传播方法。
# 定义损失函数使得预测少了的损失大，于是模型应该偏向多的方向预测。
# tf.greater询问y>y_吗？则为前面一个，否则为后面一个
# tf.reduce_sum:将所有的损失求和
loss = tf.reduce_sum(tf.where(tf.greater(y, y_), (y - y_)*COST, (y_ - y)*PROFIT))
train_step = tf.train.GradientDescentOptimizer(0.001).minimize(loss)

# 3生成会话，训练STEPS轮。
with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    STEPS = 3000
    for i in range(STEPS):
        start = (i*BATCH_SIZE) % 32
        end = (i*BATCH_SIZE) % 32 + BATCH_SIZE
        sess.run(train_step, feed_dict={x: X[start:end], y_: Y[start:end]})
        if i % 500 == 0:
            print("After %d training steps, w1 is: " % (i))
            print (sess.run(w1), "\n")
    print("Final w1 is: \n", sess.run(w1))


After 0 training steps, w1 is: 
[[-0.762993 ]
 [ 1.5095658]] 

After 500 training steps, w1 is: 
[[1.0235443]
 [1.0463371]] 

After 1000 training steps, w1 is: 
[[1.0174844]
 [1.0406414]] 

After 1500 training steps, w1 is: 
[[1.0211805]
 [1.0472372]] 

After 2000 training steps, w1 is: 
[[1.0179386]
 [1.041272 ]] 

After 2500 training steps, w1 is: 
[[1.0205938]
 [1.0390443]] 

Final w1 is: 
 [[1.0296593]
 [1.0484141]]


* 从上面的代码就可以看出，两个参数均增大了，那么整体的预测值就会增大。从而达到目的。
* 下面是单位酸奶成本高于利润的案例。

In [7]:
# coding:utf-8
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
# 酸奶成本9元， 酸奶利润1元
# 预测多了损失大，故不要预测多，故生成的模型会少预测一些
# 0导入模块，生成数据集
import tensorflow as tf
import numpy as np
BATCH_SIZE = 8
SEED = 23455
COST = 9     # 成本
PROFIT = 1   # 利润

rdm = np.random.RandomState(SEED)
X = rdm.rand(32,2)
Y = [[x1+x2+(rdm.rand()/10.0-0.05)] for (x1, x2) in X]

# 1定义神经网络的输入、参数和输出，定义前向传播过程。
x = tf.placeholder(tf.float32, shape=(None, 2))
y_ = tf.placeholder(tf.float32, shape=(None, 1))
w1= tf.Variable(tf.random_normal([2, 1], stddev=1, seed=1))
y = tf.matmul(x, w1)

# 2定义损失函数及反向传播方法。
# 重新定义损失函数，使得预测多了的损失大，于是模型应该偏向少的方向预测。
loss = tf.reduce_sum(tf.where(tf.greater(y, y_), (y - y_)*COST, (y_ - y)*PROFIT))
train_step = tf.train.GradientDescentOptimizer(0.001).minimize(loss)

# 3生成会话，训练STEPS轮。
with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    STEPS = 3000
    for i in range(STEPS):
        start = (i*BATCH_SIZE) % 32
        end = (i*BATCH_SIZE) % 32 + BATCH_SIZE
        sess.run(train_step, feed_dict={x: X[start:end], y_: Y[start:end]})
        if i % 500 == 0:
            print("After %d training steps, w1 is: " % (i))
            print(sess.run(w1), "\n")
    print("Final w1 is: \n", sess.run(w1))

After 0 training steps, w1 is: 
[[-0.80594873]
 [ 1.4873729 ]] 

After 500 training steps, w1 is: 
[[0.8732146]
 [1.006204 ]] 

After 1000 training steps, w1 is: 
[[0.9658064 ]
 [0.96982086]] 

After 1500 training steps, w1 is: 
[[0.9645447]
 [0.9682947]] 

After 2000 training steps, w1 is: 
[[0.9602475]
 [0.9742085]] 

After 2500 training steps, w1 is: 
[[0.96100295]
 [0.9699342 ]] 

Final w1 is: 
 [[0.9600407]
 [0.9733418]]


## 3.交叉熵
* 定义:表示示两个概率分布之间的距离。
* 特点：
    * 交叉熵越大，两个概率分布距离越远， 两个概率分布越相异； 
    * 交叉熵越小，两个概率分布距离越近，两个概率分布越相似。
* 计算公式：𝐇(𝐲_ , 𝐲) = −∑𝐲_ ∗ 𝒍𝒐𝒈 𝒚
* 用 Tensorflow 函数表示为
  ```
  ce= -tf.reduce_mean(y_* tf.log(tf.clip_by_value(y, 1e-12, 1.0)))
  # 这里对输入log的值做了限制，因为概率不可能大于1
  ```
* 案例
>两个神经网络模型解决二分类问题中，已知标准答案为 y_ = (1, 0)，第一个神经网络模型预测结果为y1=(0.6, 0.4)，第二个神经网络模型预测结果为 y2=(0.8, 0.2)，判断哪个神经网络模型预测的结果更接
近标准答案。 根据交叉熵的计算公式得：
H1((1,0),(0.6,0.4)) = -(1*log0.6 + 0*log0.4) ≈ -(-0.222 + 0) = 0.222
H2((1,0),(0.8,0.2)) = -(1*log0.8 + 0*log0.2) ≈ -(-0.097 + 0) = 0.097
由于 0.222>0.097，所以预测结果 y2 与标准答案 y_更接近， y2 预测更准确。

## 4. softmax函数
* 功能：将 n 分类的 n 个输出（ y1,y2…yn） 变为满足以下概率分布要求的函数。
>∀𝐱 𝐏(𝐗 = 𝐱) ∈ [𝟎, 𝟏] 且 ∑ 𝑷𝒙 (𝑿 = 𝒙) = 𝟏
* softmax 函数表示为： 𝐬𝐨𝐟𝐭𝐦𝐚𝐱(𝒚𝒊) = 𝒆^(𝒚𝒊)/∑𝒋 𝒏 =𝟏𝒆𝒚𝒊
* softmax 函数应用： 在 n 分类中， 模型会有 n 个输出， 即 y1,y2…yn， 其中 yi 表示第 i 种情况出现的可能性大小。将 n 个输出经过 softmax 函数， 可得到符合概率分布的分类结果。
* 在 Tensorflow 中，一般让模型的输出经过 sofemax 函数， 以获得输出分类的概率分布，再与标准
答案对比， **求出交叉熵， 得到损失函数**，用如下函数实现：
```
ce = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y, labels=tf.argmax(y_, 1))
cem = tf.reduce_mean(ce)
```